In [81]:
import py_entitymatching as em
import pandas as pd

A = em.read_csv_metadata('/Users/daravinds/Documents/Projects/CS-838/Stage3-EntityMatching/CS-838-bookdepository.csv',key='id')
B = em.read_csv_metadata('/Users/daravinds/Documents/Projects/CS-838/Stage3-EntityMatching/CS-838-goodreads.csv',key='id')
#sample_A, sample_B = em.down_sample(A, B, size=500, y_param=1)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [82]:
ob = em.OverlapBlocker()
status=ob.block_tables(A, B, 'Authors', 'Authors', word_level=False, q_val=3, overlap_size=5, l_output_attrs = ['id', 'Title', 'Authors', 'Genres','Publishing Date', 'Pages','Publisher','Language'], r_output_attrs = ['id', 'Title', 'Authors', 'Genres','Publishing Date', 'Pages','Publisher','Language'])
status=ob.block_candset(status, 'Title', 'Title', rem_stop_words=True, word_level=False, q_val=3, overlap_size=6 )
len(status)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


2069

In [83]:
status.to_csv('after_blocking_tuples.csv')
S = em.sample_table(status, 500)
S.to_csv('sampled_labelled_tuples.csv')

S = em.read_csv_metadata("labelled_data.csv", 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

IJ = em.split_train_test(S, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']
I

I.to_csv('train_set.csv')
J.to_csv('test_set.csv')

# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

# Generate a set of features
F = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)
F.feature_name

# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after=['match'])

# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
                strategy='mean')

# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

Metadata file is not present in the given path; proceeding to read the csv file.
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.919206,0.904099,0.906615
1,RF,0.982213,0.896455,0.933770
2,SVM,0.942997,0.569838,0.695597
3,LinReg,0.947432,0.866567,0.892675
4,LogReg,0.933237,0.981818,0.956224


In [84]:
# ab = em.AttrEquivalenceBlocker()
# status = ab.block_tuples(sample_A.ix[0], sample_B.ix[0], 'name', 'name')
# status

In [85]:
# ob = em.OverlapBlocker()
# status=ob.block_tables(A, B, 'Title', 'Title',word_level=False, q_val=3, overlap_size=6, l_output_attrs=['Title'], r_output_attrs=['Title'])
# status

In [86]:
# Train using feature vectors from I 
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='match', show_progress=False)

L = L.fillna(0)
em.set_property(L, 'key', '_id')

# Predict on L 
predictions = dt.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

# # Evaluate the predictions
em.set_property(predictions, 'fk_ltable', 'ltable_id')
em.set_property(predictions, 'fk_rtable', 'rtable_id')
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)

Precision : 80.0% (88/110)
Recall : 92.63% (88/95)
F1 : 85.85%
False positives : 22 (out of 110 positive predictions)
False negatives : 7 (out of 140 negative predictions)


In [87]:
# Train using feature vectors from I 
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='match', show_progress=False)

L = L.fillna(0)
em.set_property(L, 'key', '_id')

# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

# # Evaluate the predictions
em.set_property(predictions, 'fk_ltable', 'ltable_id')
em.set_property(predictions, 'fk_rtable', 'rtable_id')
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)

Precision : 90.2% (92/102)
Recall : 96.84% (92/95)
F1 : 93.4%
False positives : 10 (out of 102 positive predictions)
False negatives : 3 (out of 148 negative predictions)


In [88]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='match', show_progress=False)

L = L.fillna(0)
em.set_property(L, 'key', '_id')

# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

# # Evaluate the predictions
em.set_property(predictions, 'fk_ltable', 'ltable_id')
em.set_property(predictions, 'fk_rtable', 'rtable_id')
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)

Precision : 91.11% (82/90)
Recall : 86.32% (82/95)
F1 : 88.65%
False positives : 8 (out of 90 positive predictions)
False negatives : 13 (out of 160 negative predictions)


In [89]:
# Train using feature vectors from I 
svm.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='match', show_progress=False)

L = L.fillna(0)
em.set_property(L, 'key', '_id')

# Predict on L 
predictions = svm.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
              append=True, target_attr='predicted', inplace=False)

# # Evaluate the predictions
em.set_property(predictions, 'fk_ltable', 'ltable_id')
em.set_property(predictions, 'fk_rtable', 'rtable_id')
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)

Precision : 94.44% (51/54)
Recall : 53.68% (51/95)
F1 : 68.46%
False positives : 3 (out of 54 positive predictions)
False negatives : 44 (out of 196 negative predictions)


In [90]:
# Train using feature vectors from I 
ln.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='match', show_progress=False)

L = L.fillna(0)
em.set_property(L, 'key', '_id')

# Predict on L 
predictions = ln.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

# # Evaluate the predictions
em.set_property(predictions, 'fk_ltable', 'ltable_id')
em.set_property(predictions, 'fk_rtable', 'rtable_id')
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)

There is no proba function defined for Linear Regression Matcher in scikit learn. So we return the probs as 1


Precision : 92.86% (78/84)
Recall : 82.11% (78/95)
F1 : 87.15%
False positives : 6 (out of 84 positive predictions)
False negatives : 17 (out of 166 negative predictions)
